In [39]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

# Initialize an empty DataFrame to store the filtered rows
filtered_df = pd.DataFrame(columns=["id", "text"])


# Function to check if both keywords are in the abstract
def contains_keywords(text):
    if isinstance(text, list) and len(text) > 1:
        abstract = " ".join(text[1]).lower()
        return "natural language processing" in abstract and "translation" in abstract
    return False


# Load the dataset in streaming mode
dataset = load_dataset("howey/unarXive", split="train", streaming=True)

# Initialize a counter for the total number of rows
total_rows = 0

# Initialize tqdm progress bar
progress_bar = tqdm(total=1000, desc="Processing rows")

# Buffer to store rows temporarily
buffer = []

# Iterate over the dataset in chunks
for example in dataset:
    buffer.append(example)

    # Process the buffer when it reaches 100 rows
    if len(buffer) >= 100000:
        # Convert the buffer to a DataFrame
        buffer_df = pd.DataFrame(buffer)

        # Filter the rows based on the criteria
        filtered_chunk = buffer_df[buffer_df["text"].apply(contains_keywords)]

        # Append the filtered rows to the DataFrame
        filtered_df = pd.concat([filtered_df, filtered_chunk], ignore_index=True)

        # Update the total number of rows
        total_rows += len(filtered_chunk)

        # Update the progress bar
        progress_bar.update(len(filtered_chunk))

        # Clear the buffer
        buffer = []

        # Break the loop if we have 1000 rows
        if total_rows >= 1000:
            break

# Process any remaining rows in the buffer
if buffer:
    buffer_df = pd.DataFrame(buffer)
    filtered_chunk = buffer_df[buffer_df["text"].apply(contains_keywords)]
    filtered_df = pd.concat([filtered_df, filtered_chunk], ignore_index=True)
    total_rows += len(filtered_chunk)
    progress_bar.update(len(filtered_chunk))

# Ensure the DataFrame has exactly 1000 rows
filtered_df = filtered_df.head(1000)

# Close the progress bar
progress_bar.close()

filtered_df.to_parquet("filtered_translation_data.parquet")

Processing rows:  14%|█▍        | 142/1000 [2:01:11<14:42:00, 61.68s/it]HTTP Error 504 thrown while requesting GET https://huggingface.co/datasets/howey/unarXive/resolve/daf6d547b5290c0f783a6bc8573078c0a60771cd/data/train-00084-of-00198-2667d4e6238c850b.parquet
Retrying in 1s [Retry 1/5].
Processing rows:  36%|███▌      | 355/1000 [5:28:45<9:57:18, 55.56s/it] 
